In [1]:
# coding: utf-8

import matplotlib.pyplot as plt
import pandas as pd

from files.base import Base

In [2]:
df = pd.read_csv(Base.BASE, sep=";", names=['article','type','positive','negative','neutral','link','origin'])
df.head(2)

,article,type,positive,negative,neutral,link,origin
0,Dilma deve discutir propostas com servidores e...,impartial,1,-2,-1,https://www.opovo.com.br/noticias/politica/201...,O Povo
1,Como a aprovação do Orçamento está opondo Gued...,partial,3,-2,1,https://diariodonordeste.verdesmares.com.br/po...,Diário do Nordeste


In [36]:
db = pd.DataFrame({
    'total' : {
        'Total': df.article.count(),
        'Diário do Nordeste': df[(df.origin == "Diário do Nordeste")].article.count(),
        'G1 Ceará': df[(df.origin == "G1 Ceará")].article.count(),
        'O Povo': df[(df.origin == "O Povo")].article.count()
    },
    'impartial' : {
        'Total': df[df.type == "impartial"].article.count(),
        'Diário do Nordeste': df[(df.origin == "Diário do Nordeste") & (df.type == "impartial")].article.count(),
        'G1 Ceará': df[(df.origin == "G1 Ceará") & (df.type == "impartial")].article.count(),
        'O Povo': df[(df.origin == "O Povo") & (df.type == "impartial")].article.count()
    },
    'partial' : {
        'Total':  df[df.type == "partial"].article.count(),
        'Diário do Nordeste': df[(df.origin == "Diário do Nordeste") & (df.type == "partial")].article.count(),
        'G1 Ceará': df[(df.origin == "G1 Ceará") & (df.type == "partial")].article.count(),
        'O Povo': df[(df.origin == "O Povo") & (df.type == "partial")].article.count(),
    },
})

db

,total,impartial,partial
Total,11095,4473,6622
Diário do Nordeste,2042,639,1403
G1 Ceará,1866,539,1327
O Povo,7187,3295,3892


1403